In [1]:
pip install quandl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import quandl as q

In [3]:
#set the API key
q.ApiConfig.api_key ='_Fjzwen5ERibsCY-wTsn'

In [6]:
headers = {
    "Authorization": "_Fjzwen5ERibsCY-wTsn"
}

response = requests.get(url, params=params, headers=headers)


In [7]:
import requests

# Define the API endpoint and parameters
url = "https://api.finage.co.uk/last/stock/msft"
params = {
    "symbol": "MSFT",
    "from": "2014-01-01",
    "to": "2023-01-01"
}

# Send GET request and handle the response
response = requests.get(url, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    # Process the data as needed
    print(data)
else:
    print("Error occurred. Status code:", response.status_code)


Error occurred. Status code: 401


In [4]:
#send a get request to query Microsoft's end of day stock prices from 1st #Jan, 2010 to 1st Jan, 2019

msft_data = q.get('EOD/MSFT', start_date='2014-01-01', end_date='2023-01-01')

LimitExceededError: (Status 429) (Quandl Error QELx06) You have exceeded the API speed limit and your account has temporaly been disabled.  Please contact clientsuccess@nasdaq.com for more information.

In [ ]:
msft_data.head()

In [ ]:
msft_data.info()

As seen in the screenshot above, the DataFrame contains DatetimeIndex, which means we’re dealing with time-series data.

An index can be thought of as a data structure that helps us modify or reference the data. Time-series data is a sequence of snapshots of prices taken at consecutive, equally spaced intervals of time.

In trading, EOD stock pricing data captures the movement of certain parameters about a stock, such as the stock price, over a specified period of time with data points recorded at regular intervals.

Important Terminology
Looking at other columns, let’s try to understand what each column represents:

Open/Close — Captures the opening/closing price of the stock

Adj_Open/Adj_Close — An adjusted opening/closing price is a stock’s price on any given day of trading that has been revised to include any dividend distributions, stock splits, and other corporate actions that occurred at any time before the next day’s open.

Volume — It records the number of shares that are being traded on any given day of trading.

High/Low — It tracks the highest and the lowest price of the stock during a particular day of trading.

These are the important columns that we will focus on at this point in time.

We can learn about the summary statistics of the data, which shows us the number of rows, mean, max, standard deviations, and so on. 
Try running the following line of code in the Ipython cell:

In [ ]:
msft_data.describe()

resample()

Pandas’ resample() method is used to facilitate control and flexibility on the frequency conversion of the time series data. We can specify the time intervals to resample the data to monthly, quarterly, or yearly, and perform the required operation over it.

In [ ]:
msft_data.resample('M').mean()

Calculating returns

A financial return is simply the money made or lost on an investment. A return can be expressed nominally as the change in the amount of an investment over time. It can be calculated as the percentage derived from the ratio of profit to investment.

We have the pct_change() at our disposal for this purpose. Here is how you can calculate returns:

In [ ]:
# assign `Adj Close` to `daily_close`
daily_close = msft_data[['Adj_Close']]

In [ ]:
# returns as fractional change
daily_return = daily_close.pct_change()

In [ ]:
# replacing NA values with 0

daily_return.fillna(0, inplace=True)

In [ ]:
print(daily_return)

In [ ]:
#Now, to calculate monthly returns, all you need to do is:

mdata = msft_data.resample('M').apply(lambda x: x[-1])

monthly_return = mdata.pct_change()

After resampling the data to months (for business days), we can get the last day of trading in the month using the apply() function.

apply() takes in a function and applies it to each and every row of the Pandas series. The lambda function is an anonymous function in Python which can be defined without a name, and only takes expressions in the following format:

The concept of moving averages is going to build the base for our momentum-based trading strategy.

In finance, analysts often have to evaluate statistical metrics continually over a sliding window of time, which is called moving window calculations.

Let’s see how we can calculate the rolling mean over a window of 50 days, and slide the window by 1 day.

rolling()
This is the magical function which does the tricks for us:

In [ ]:
# assigning adjusted closing prices to 

adj_price = msft_data['Adj_Close']

In [ ]:
# calculate the moving average
mav = adj_price.rolling(window=50).mean()

In [ ]:
mav[-10:].head(5)

You’ll see the rolling mean over a window of 50 days (approx. 2 months). Moving averages help smooth out any fluctuations or spikes in the data, and give you a smoother curve for the performance of the company.

We can plot and see the difference:

In [ ]:
import matplotlib.pyplot as plt

adj_price.plot()

In [ ]:
#You can now plot the rolling mean():

mav.plot()

Here comes the final and most interesting part: designing and making the trading strategy. This will be a step-by-step guide to developing a momentum-based Simple Moving Average Crossover (SMAC) strategy.

Momentum-based strategies are based on a technical indicator that capitalizes on the continuance of the market trend. We purchase securities that show an upwards trend and short-sell securities which show a downward trend.

The SMAC strategy is a well-known schematic momentum strategy. It is a long-only strategy. Momentum, here, is the total return of stock including the dividends over the last n months. This period of n months is called the lookback period.

There are 3 main types of lookback periods: short term, intermediate-term, and long term. We need to define 2 different lookback periods of a particular time series.

A buy signal is generated when the shorter lookback rolling mean (or moving average) overshoots the longer lookback moving average. A sell signal occurs when the shorter lookback moving average dips below the longer moving average.

Now, let’s see how the code for this strategy will look:

In [ ]:
# step1: initialize the short and long lookback periods

short_lb = 50 
long_lb = 120

In [ ]:
# step2: initialize a new DataFrame called signal_df with a signal column

signal_df = pd.DataFrame(index=msft_data.index)
signal_df['signal'] = 0.0

In [ ]:
# step3: create a short simple moving average over the short lookback period
signal_df['short_mav'] = msft_data['Adj_Close'].rolling(window=short_lb, min_periods=1, center=False).mean()

In [ ]:
# step4: create long simple moving average over the long lookback period

signal_df['long_mav'] = msft_data['Adj_Close'].rolling(window=long_lb, min_periods=1, center=False).mean()

In [ ]:
# step5: generate the signals based on the conditional statement

signal_df['signal'][short_lb:] = np.where(signal_df['short_mav'][short_lb:] > signal_df['long_mav'][short_lb:], 1.0, 0.0)   

In [ ]:
# step6: create the trading orders based on the positions column

signal_df['positions'] = signal_df['signal'].diff()
signal_df[signal_df['positions'] == -1.0]

Let’s see what’s happening here. We have created 2 lookback periods. The short lookback period short_lb is 50 days, and the longer lookback period for the long moving average is defined as a long_lb of 120 days.

We have created a new DataFrame which is designed to capture the signals. These signals are being generated whenever the short moving average crosses the long moving average using the np.where. It assigns 1.0 for true and 0.0 if the condition comes out to be false.

The positions columns in the DataFrame tells us if there is a buy signal or a sell signal, or to stay put. We're basically calculating the difference in the signals column from the previous row using diff.

And there we have our strategy implemented in just 6 steps using Pandas. Easy, wasn't it?

Now, let’s try to visualize this using Matplotlib. All we need to do is initialize a plot figure, add the adjusted closing prices, short, and long moving averages to the plot, and then plot the buy and sell signals using the positions column in the signal_df above:

In [ ]:
# initialize the plot using plt
fig = plt.figure()

# Add a subplot and label for y-axis
plt1 = fig.add_subplot(111,  ylabel='Price in $')
msft_data['Adj_Close'].plot(ax=plt1, color='r', lw=2.)
# plot the short and long lookback moving averages
signal_df[['short_mav', 'long_mav']].plot(ax=plt1, lw=2., figsize=(12,8))
# plotting the sell signals
plt1.plot(signal_df.loc[signal_df.positions == -1.0].index, signal_df.short_mav[signal_df.positions == -1.0],'v', markersize=10, color='k')
# plotting the buy signals
plt1.plot(signal_df.loc[signal_df.positions == 1.0].index, signal_df.short_mav[signal_df.positions == 1.0],'^', markersize=10, color='m')        # Show the plotplt.show()

plt.legend()
plt.show()


In [ ]:
print(3*'un'+ 'ium')